In [11]:
import pandas as pd
pd.set_option("display.max_colwidth", None)   # <-- añade esto una vez al inicio del notebook

import numpy as np
import unicodedata
from pathlib import Path


In [12]:

# --- helpers ---
def normalize_text(s: str) -> str:
    if pd.isna(s): 
        return ""
    s = str(s).lower()
    s = unicodedata.normalize("NFKD", s)
    s = "".join(ch for ch in s if unicodedata.category(ch) != "Mn")  # quita acentos
    return s

def load_clean(path, store_name):
    df = pd.read_csv(path, encoding="utf-8-sig")
    # columnas esperadas: name, price (€), price_per_kg (como generaste en los merges)
    # por si hay variantes:
    cols = {c.lower(): c for c in df.columns}
    name_col  = cols.get("name", "name")
    price_col = "price (€)" if "price (€)" in df.columns else cols.get("price", None)
    ppk_col   = "price_per_kg" if "price_per_kg" in df.columns else cols.get("price_per_kg", None)

    # coerción segura
    df = df.rename(columns={name_col:"name"})
    if price_col and price_col != "price (€)":
        df = df.rename(columns={price_col:"price (€)"})
    if ppk_col and ppk_col != "price_per_kg":
        df = df.rename(columns={ppk_col:"price_per_kg"})

    df["price (€)"] = pd.to_numeric(df["price (€)"], errors="coerce")
    if "price_per_kg" in df:
        df["price_per_kg"] = pd.to_numeric(df["price_per_kg"], errors="coerce")
    else:
        df["price_per_kg"] = np.nan

    df["store"] = store_name
    df["name_norm"] = df["name"].map(normalize_text)
    return df[["store","name","name_norm","price (€)","price_per_kg"]]



In [13]:

# --- carga los dos CSV ya “merged” ---
df_dia     = load_clean(r"C:\Users\gerar\Desktop\random\web-scraping\Supermercats\Data\DIA\dia_merged_clean.csv", "DIA")
df_bonpreu = load_clean(r"C:\Users\gerar\Desktop\random\web-scraping\Supermercats\Data\Bonpreu\bonpreu_merged_clean.csv", "Bonpreu")

df_all = pd.concat([df_dia, df_bonpreu], ignore_index=True)


In [18]:


def best_for(term: str):
    term_norm = normalize_text(term)
    sub = df_all[df_all["name_norm"].str.contains(term_norm, na=False)].copy()

    if sub.empty:
        return pd.DataFrame([{"criterio":"sin coincidencias","store":None,"name":"(ninguno)"}])

    out = []

    # --- precio absoluto ---
    sub_price = sub.dropna(subset=["price (€)"])
    if not sub_price.empty:
        r_min = sub_price.loc[sub_price["price (€)"].idxmin()]
        r_max = sub_price.loc[sub_price["price (€)"].idxmax()]
        out += [
            {
                "criterio": "mínimo precio",
                "store": r_min["store"],
                "name": r_min["name"],
                "price (€)": round(float(r_min["price (€)"]), 2),
                "price_per_kg": (None if pd.isna(r_min["price_per_kg"])
                                 else round(float(r_min["price_per_kg"]), 2)),
            },
            {
                "criterio": "máximo precio",
                "store": r_max["store"],
                "name": r_max["name"],
                "price (€)": round(float(r_max["price (€)"]), 2),
                "price_per_kg": (None if pd.isna(r_max["price_per_kg"])
                                 else round(float(r_max["price_per_kg"]), 2)),
            },
        ]

    # --- precio por kg ---
    sub_ppk = sub.dropna(subset=["price_per_kg"])
    if not sub_ppk.empty:
        r_min_ppk = sub_ppk.loc[sub_ppk["price_per_kg"].idxmin()]
        r_max_ppk = sub_ppk.loc[sub_ppk["price_per_kg"].idxmax()]
        out += [
            {
                "criterio": "mínimo €/kg",
                "store": r_min_ppk["store"],
                "name": r_min_ppk["name"],
                "price (€)": round(float(r_min_ppk["price (€)"]), 2),
                "price_per_kg": round(float(r_min_ppk["price_per_kg"]), 2),
            },
            {
                "criterio": "máximo €/kg",
                "store": r_max_ppk["store"],
                "name": r_max_ppk["name"],
                "price (€)": round(float(r_max_ppk["price (€)"]), 2),
                "price_per_kg": round(float(r_max_ppk["price_per_kg"]), 2),
            },
        ]

    if not out:
        return pd.DataFrame([{"criterio":"sin precios válidos","store":None,"name":"(ninguno)"}])

    # orden más agradable
    order = {"mínimo precio": 0, "máximo precio": 1, "mínimo €/kg": 2, "máximo €/kg": 3}
    df = pd.DataFrame(out)
    return df.sort_values("criterio", key=lambda s: s.map(order)).reset_index(drop=True)


In [19]:
# --- ejemplo de consulta ---
gnocchi=best_for("gnocchi")

In [22]:
gnocchi

,criterio,store,name,price (€),price_per_kg
0,mínimo precio,DIA,Gnocchi de patata Selección Mundial de Dia bandeja 500 g,1.10,2.2
1,máximo precio,Bonpreu,SCHÄR Gnocchi sense gluten,3.99,13.3
2,mínimo €/kg,DIA,Gnocchi de patata Selección Mundial de Dia bandeja 500 g,1.10,2.2
3,máximo €/kg,Bonpreu,SCHÄR Gnocchi sense gluten,3.99,13.3


In [21]:
best_for("arroz basmati")

,criterio,store,name,price (€),price_per_kg
0,mínimo precio,DIA,Arroz basmati Nomen vaso 2 x 125 g,1.25,5.00
1,máximo precio,DIA,Arroz basmati Tilda paquete 1 Kg,3.59,3.59
2,mínimo €/kg,DIA,Arroz basmati Selección Mundial de Dia paquete 1 Kg,2.20,2.20
3,máximo €/kg,DIA,Arroz basmati Brillante vaso 400 g,2.43,6.08
